In [75]:
import chdb
import time
import os


def get_directory_size_mb(directory):
    """
    Get the total size of a directory and its contents in megabytes.
    
    Args:
        directory (str): Path to the directory
        
    Returns:
        float: Size of the directory in megabytes
    """
    total_size = 0
    
    try:
        # Walk through the directory
        for root, dirs, files in os.walk(directory):
            # Sum up the size of all files in current directory
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    # Use os.path.getsize() to get file size
                    total_size += os.path.getsize(file_path)
                except (OSError, FileNotFoundError):
                    # Handle cases where file can't be accessed
                    print(f"Could not access file: {file_path}")
                    continue
                    
    except PermissionError:
        print(f"Permission denied: Cannot access {directory}")
    except Exception as e:
        print(f"Error accessing {directory}: {e}")
    
    # Convert bytes to megabytes and round to 2 decimal places
    size_mb = round(total_size / (1024 * 1024), 2)
    return size_mb

def count_files(directory):
    """
    Recursively count the number of files in a directory and its subdirectories.
    
    Args:
        directory (str): Path to the directory to count files in
        
    Returns:
        int: Total number of files found
    """
    total_files = 0
    
    try:
        # Iterate through all items in the directory
        for item in os.scandir(directory):
            if item.is_file():
                # If it's a file, increment the counter
                total_files += 1
            elif item.is_dir():
                # If it's a directory, recursively count files inside it
                total_files += count_files(item.path)
                
    except PermissionError:
        print(f"Permission denied: Cannot access {directory}")
    except Exception as e:
        print(f"Error accessing {directory}: {e}")
        
    return total_files

def count_folders(directory):
    """
    Recursively count the number of folders in a directory and its subdirectories.
    
    Args:
        directory (str): Path to the directory to count folders in
        
    Returns:
        int: Total number of folders found
    """
    total_folders = 0
    
    try:
        # Iterate through all items in the directory
        for item in os.scandir(directory):
            if item.is_dir():
                # If it's a directory, increment counter and recursively count subdirectories
                total_folders += 1
                total_folders += count_folders(item.path)
                
    except PermissionError:
        print(f"Permission denied: Cannot access {directory}")
    except Exception as e:
        print(f"Error accessing {directory}: {e}")
        
    return total_folders


# Instructions
1. Init project: `uv sync`
2. Generate data: `uv run python generator.py`

Generating 12 months of data will get you roughly 27,079,811 bytes (354.9 MB on disk) for 139,001 items in your `ecommerce_data` directory




In [76]:
data_dir = "ecommerce_data/**/*.json"

data_loc = f"file('{data_dir}','JSONLines')"

# This describes the data in the directory
chdb.query(f"DESCRIBE {data_loc} SETTINGS describe_compact_output=1")



"event_time","Nullable(DateTime)"
"timestamp","Nullable(DateTime)"
"product_id","Nullable(String)"
"product_name","Nullable(String)"
"category","Nullable(String)"
"quantity_sold","Nullable(Int64)"
"unit_price","Nullable(Float64)"
"total_revenue","Nullable(Float64)"
"weekday","Nullable(String)"
"is_weekend","Nullable(Bool)"
"hour","Nullable(Int64)"

In [ ]:
# Lets compare the size of data in each directory
# Number of files in each directory recursively

# Category First
data_dir = "category_first/ecommerce_data/"

num_files = count_files(data_dir)
num_folders = count_folders(data_dir)
size = get_directory_size_mb(data_dir)
print(f"Number of files in {data_dir}: {num_files}")
print(f"Number of folders in {data_dir}: {num_folders}")
print("Total Number of files and folders: ", num_files + num_folders)
print(f"Size of {data_dir}: {size}")

# Category Last
data_dir = "ecommerce_data/"

num_files = count_files(data_dir)
num_folders = count_folders(data_dir)
size = get_directory_size_mb(data_dir)
print(f"Number of files in {data_dir}: {num_files}")
print(f"Number of folders in {data_dir}: {num_folders}")
print("Total Number of files and folders: ", num_files + num_folders)
print(f"Size of {data_dir}: {size}")





Number of files in category_first/ecommerce_data/: 94100
Number of folders in category_first/ecommerce_data/: 45205
Total Number of files and folders:  139305
Size of category_first/ecommerce_data/: 31.64


In [20]:
# Now lets get all the different products
chdb.query(f"SELECT DISTINCT product_name FROM {data_loc}")

"Secured responsive Graphical User Interface"
"Down-sized didactic algorithm"
"Configurable radical forecast"
"Progressive user-facing analyzer"
"Robust leadingedge analyzer"
"Future-proofed discrete initiative"
"Progressive 24hour emulation"
"Total maximized firmware"
"Centralized zero-defect archive"
"Centralized composite knowledgebase"

In [ ]:
# Now lets get the sales in february by using a query

feb_time_query_start_time = time.time()
result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} where toMonth(event_time) = 2 GROUP BY category")
feb_time_query_end_time = time.time()
feb_time_query_time = feb_time_query_end_time - feb_time_query_start_time
print(f"Time taken: {feb_time_query_time} seconds")

print("Sales by category in February 2025")
print(result)


Time taken: 7.154558897018433 seconds
Sales by category in February 2025
"Books",4818
"Sports",13639
"Clothing",9138
"Home & Garden",4647
"Electronics",14013



In [22]:
# Lets get the sales in febrary by using globs
data_dir = "ecommerce_data/2025/02/**/*.json"

data_loc = f"file('{data_dir}','JSONLines')"

feb_time_globs_start_time = time.time()
result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} GROUP BY category")
feb_time_globs_end_time = time.time()
feb_time_globs_time = feb_time_globs_end_time - feb_time_globs_start_time
print(f"Time taken: {feb_time_globs_time} seconds")

print("Sales by category in February 2025")
print(result)

Time taken: 0.4534158706665039 seconds
Sales by category in February 2025
"Books",4656
"Sports",13164
"Clothing",8788
"Home & Garden",4456
"Electronics",13563



In [33]:
# Lets get all the Electronics sales the inefficient way
all_1_cat_ineffecient_start_time = time.time()
result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} where category = 'Electronics' GROUP BY category")
all_1_cat_ineffecient_end_time = time.time()
all_1_cat_ineffecient_time = all_1_cat_ineffecient_end_time - all_1_cat_ineffecient_start_time
print(f"Time taken: {all_1_cat_ineffecient_time} seconds")

print("Sales by category in February 2025")
print(result)

Time taken: 2.282703161239624 seconds
Sales by category in February 2025
"Electronics",203292



In [ ]:
# Lets get all the Electronics sales
data_dir = "ecommerce_data/*/*/*/*/electronics/*.json"

data_loc = f"file('{data_dir}','JSONLines')"

cat_last_start_time = time.time()
result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} GROUP BY category")
cat_last_end_time = time.time()
cat_last_time = cat_last_end_time - cat_last_start_time
print(f"Time taken: {cat_last_time} seconds")

print("Sales by category(electronics)")
print(result)


Time taken: 3.3393590450286865 seconds
Sales by category(electronics)
"Electronics",203292



In [54]:
# Lets get all the sales by category for the month of february for category first
data_dir = "category_first/ecommerce_data/**/2025/02/**/*.json"
feb_cat_first_start_time = time.time()
data_loc = f"file('{data_dir}','JSONLines')"
feb_cat_first_end_time = time.time()
feb_cat_first_time = feb_cat_first_end_time - feb_cat_first_start_time
print(f"Time taken: {feb_cat_first_time} seconds")

result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} GROUP BY category")

print("Sales by category(electronics) in February 2025")
print(result)


Time taken: 3.314018249511719e-05 seconds
Sales by category(electronics) in February 2025
"Books",8574
"Sports",9316
"Clothing",13424
"Home & Garden",26246
"Electronics",31047



In [ ]:
# Lets compare with the category first
data_dir =  "category_first/ecommerce_data/electronics/**/*.json"
data_loc = f"file('{data_dir}','JSONLines')"

cat_first_start_time = time.time()

chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} GROUP BY category")
cat_first_end_time = time.time()
print(f"Time taken: {cat_first_end_time - cat_first_start_time} seconds")

print("Sales by category(electronics) in February 2025")
print(result)


sports_start_time = time.time()
data_dir = "category_first/ecommerce_data/sports/**/*.json"
data_loc = f"file('{data_dir}','JSONLines')"
sports_result = chdb.query(f"SELECT category, SUM(quantity_sold) as total_sales FROM {data_loc} GROUP BY category")
sports_end_time = time.time()
print(f"Time taken: {sports_end_time - sports_start_time} seconds")

print("Sales by category(sports) in February 2025")
print(sports_result)


Time taken: 1.1165568828582764 seconds
Sales by category(electronics) in February 2025
"Electronics",203292

Time taken: 1.3575620651245117 seconds
Sales by category(sports) in February 2025
"Sports",135597

